In [1]:
from datasets import load_dataset

# Load the dataset from the JSONL file
dataset = load_dataset('json', data_files='user_queries.jsonl')

# Convert the dataset to a pandas DataFrame
df = dataset['train'].to_pandas()

df.head()

,conversation_hash,first_query
0,34e6bc4f50a9cd0c5a5f947ddbb9824b,"Hi there, Can you help me revise my love story..."
1,e1dda5bd84cb41a924308748c112243c,[VARIABLE] = I want a very fast epic final bos...
2,7f780423584ccd13562f838ca9496b1d,"You, ChatGPT, will be my prompt engineer. We w..."
3,d9dbe9e0356aa594d9d20adb65f7ed3a,What is the derivative of ln((x+a)/(x+b)) with...
4,fb9210f01518ec83a7203ea51a23fc4c,can you predict the next thing in the followin...


In [ ]:
import os
from dotenv import load_dotenv
import requests
import json

load_dotenv()

output = []

# Iterate through each query in the DataFrame
for index, row in df.iterrows():
    query = row['first_query']
    print(str(index) + " " + query)

    output.append({
        "role": "user",
        "content": query
    })

    url = "https://api.fireworks.ai/inference/v1/chat/completions"
    payload = {
    "model": "accounts/fireworks/models/deepseek-r1",
    "max_tokens": 20480,
    "top_p": 1,
    "top_k": 40,
    "presence_penalty": 0,
    "frequency_penalty": 0,
    "temperature": 0.6,
    "messages": [
        {
        "role": "user",
        "content": query
        }
    ]
    }
    headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.getenv("FIREWORKS_API_KEY")
    }
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
    res = response.json()['choices'][0]['message']['content']
    output.append({
        "role": "assistant",
        "content": res
    })

    with open('reasoning-messages.jsonl', 'a') as f:
        json.dump({"messages": output}, f)
        f.write('\n')


0 Hi there, Can you help me revise my love story for my application of an open work permit to accompany my wife in her study in Canada?
1 [VARIABLE] = I want a very fast epic final boss battle song with a very complex and banger melody in the style of hardcore
Write music in abc notation that can be played on a website. ALWAYS make it about [VARIABLE]. Piano is the instrument. Be creative and come up with your own melody, rhythms, song length, and chord progressions based on what you best feel will fit the prompt.
RULES:
ONLY OUTPUT ONE SONG.
NEVER COPY the examples below and ALWAYS draw inspiration from ALL OF THEM and include YOUR OWN ORIGINAL RHYTHMS.
ALWAYS include multiple lengths of note
Think about real songs and ALWAYS draw out melodies and rhythms from them
ALTERNATE between long and short notes, make syncopated rhythms
At the end ALWAYS give a short description of what you wrote

FOCUS ON INCREASING THE COMPLEXITY OF THE SONG AND NEVER JUST HAVE THE SAME NOTE PLAY OVER AND OV